In [1]:
import tensorflow as tf
import numpy as np
import time

In [2]:
phn_61 = ['aa', 'ae', 'ah', 'ao', 'aw', 'ax', 'ax-h', 'axr', 'ay', 'b', 'bcl', 'ch', 'd', 'dcl', 'dh', 'dx', 'eh', 'el', 'em', 'en', 'eng', 'epi', 'er', 'ey', 'f', 'g', 'gcl', 'h#', 'hh', 'hv', 'ih', 'ix', 'iy', 'jh', 'k', 'kcl', 'l', 'm', 'n', 'ng', 'nx', 'ow', 'oy', 'p', 'pau', 'pcl', 'q', 'r', 's', 'sh', 't', 'tcl', 'th', 'uh', 'uw', 'ux', 'v', 'w', 'y', 'z', 'zh']
phn_39 = ['ae', 'ao', 'aw', 'ax', 'ay', 'b', 'ch', 'd', 'dh', 'dx', 'eh', 'er', 'ey', 'f', 'g', 'h#', 'hh', 'ix', 'iy', 'jh', 'k', 'l', 'm', 'n', 'ng', 'ow', 'oy', 'p', 'r', 's', 't', 'th', 'uh', 'uw', 'v', 'w', 'y', 'z', 'zh']
mapping = {'ah': 'ax', 'ax-h': 'ax', 'ux': 'uw', 'aa': 'ao', 'ih': 'ix', 'axr': 'er', 'el': 'l', 'em': 'm', 'en': 'n', 'nx': 'n', 'eng': 'ng', 'sh': 'zh', 'hv': 'hh', 'bcl': 'h#', 'pcl': 'h#', 'dcl': 'h#', 'tcl': 'h#', 'gcl': 'h#', 'kcl': 'h#', 'q': 'h#', 'epi': 'h#', 'pau': 'h#'}

TRAIN_FILE = './data/fbank/train.tfrecords'
DEV_FILE = './data/fbank/dev.tfrecords'
TEST_FILE = './data/fbank/test.tfrecords'
checkpoints_path = './model/seq2seq_pyramidal/ckpt'

feat_type = 'fbank'
feats_dim = 39 if feat_type=='mfcc' else 123
labels_sos_id = len(phn_61)
labels_eos_id = len(phn_61) + 1
num_classes = len(phn_61) + 2

num_unit_encoder = 128
num_unit_decoder = 256
learning_rate = 0.001
n_hidden_layer = 3
beam_width = 10
batch_size = 64
epochs = 100

In [3]:
class Model(object):
    def __init__(self, batch_size, num_unit_encoder, num_unit_decoder, n_hidden_layer, feats_dim, num_classes, labels_sos_id, labels_eos_id,
                 learning_rate=0.001, beam_width=10, phn_61=None, phn_39=None, mapping=None, file_type=None, model_type=None):
        iterator = self._get_iterator(batch_size, feats_dim, labels_sos_id, labels_eos_id, file_type, model_type)
        self.iterator_initializer = iterator.initializer
        
        batched_data = iterator.get_next()
        features = batched_data[0]
        labels_in = batched_data[1]
        labels_out = batched_data[2]
        feats_seq_len = tf.to_int32(batched_data[3])
        labels_in_seq_len = tf.to_int32(batched_data[4])
        
        decoded = self._compute_dynamic_decode(features, labels_in, feats_seq_len, labels_in_seq_len, num_unit_encoder,
                                               num_unit_decoder, n_hidden_layer, num_classes, beam_width, model_type)
        if model_type == 'train':
            logits = decoded
            self.loss = self._compute_loss(labels_out, labels_in_seq_len, logits)
            self.update_step = self._get_update_step(self.loss, learning_rate)
        elif model_type == 'eval':
            logits = decoded
            self.loss = self._compute_loss(labels_out, labels_in_seq_len, logits)
        else: # model_type == 'infer'
            predicted_ids = decoded
            self.per = self._compute_per(predicted_ids, labels_out, labels_eos_id, phn_61, phn_39, mapping)
        
        self.saver = tf.train.Saver()
            
    def _get_sparse_tensor(self, dense, default):
        indices = tf.to_int64(tf.where(tf.not_equal(dense, default)))
        vals = tf.to_int32(tf.gather_nd(dense, indices))
        shape = tf.to_int64(tf.shape(dense))
        return tf.SparseTensor(indices, vals, shape)
        
    def _get_iterator(self, batch_size, feats_dim, labels_sos_id, labels_eos_id, file_type, model_type):
        dataset = tf.contrib.data.TFRecordDataset(file_type)
        context_features = {'feats_seq_len': tf.FixedLenFeature([], dtype=tf.int64),
                           'labels_seq_len': tf.FixedLenFeature([], dtype=tf.int64)}
        sequence_features = {'features': tf.FixedLenSequenceFeature([feats_dim], dtype=tf.float32),
                            'labels': tf.FixedLenSequenceFeature([], dtype=tf.int64)}
        dataset = dataset.map(lambda serialized_example: tf.parse_single_sequence_example(serialized_example,
                                                                                         context_features=context_features,
                                                                                         sequence_features=sequence_features))
        dataset = dataset.map(lambda context, sequence: (sequence['features'], sequence['labels'], 
                                                         context['feats_seq_len'], context['labels_seq_len']))
        dataset = dataset.map(lambda features, labels, feats_seq_len, labels_seq_len: (features, 
                                                        tf.concat(([labels_sos_id], labels),0),
                                                        tf.concat((labels, [labels_eos_id]), 0),
                                                                feats_seq_len, labels_seq_len))
        dataset = dataset.map(lambda features, labels_in, labels_out, feats_seq_len, labels_seq_len: 
                                          (features, labels_in, labels_out, feats_seq_len, tf.size(labels_in, out_type=tf.int64)))
        def batching_func(x):
            return x.padded_batch(batch_size,
                                 padded_shapes=(tf.TensorShape([None, feats_dim]),
                                               tf.TensorShape([None]),
                                               tf.TensorShape([None]),
                                               tf.TensorShape([]),
                                               tf.TensorShape([])),
                                 padding_values=(tf.cast(0, tf.float32),
                                                tf.cast(labels_eos_id, tf.int64),
                                                tf.cast(labels_eos_id, tf.int64),
                                                tf.cast(0, tf.int64),
                                                tf.cast(0, tf.int64)))
        def key_func(features, labels_in, labels_out, feats_seq_len, labels_in_seq_len):
            f0 = lambda: tf.constant(0, tf.int64)
            f1 = lambda: tf.constant(1, tf.int64)
            f2 = lambda: tf.constant(2, tf.int64)
            f3 = lambda: tf.constant(3, tf.int64)
            f4 = lambda: tf.constant(4, tf.int64)
            f5 = lambda: tf.constant(5, tf.int64)
            f6 = lambda: tf.constant(6, tf.int64)
            
            return tf.case([(tf.less_equal(feats_seq_len, 200), f0),
                   (tf.less_equal(feats_seq_len, 250), f1),
                   (tf.less_equal(feats_seq_len, 300), f2),
                   (tf.less_equal(feats_seq_len, 350), f3),
                   (tf.less_equal(feats_seq_len, 400), f4),
                   (tf.less_equal(feats_seq_len, 500), f5)], default=f6)
        
        def reduce_func(bucket_id, windowed_data):
            return batching_func(windowed_data)
        
        if model_type=='train':
            dataset = dataset.shuffle(10000)
            batched_dataset = dataset.group_by_window(key_func=key_func, reduce_func=reduce_func, window_size=batch_size)
            batched_dataset = batched_dataset.shuffle(10000)
        else:
            batched_dataset = batching_func(dataset)
            
        return batched_dataset.make_initializable_iterator()
    
    def _compute_encoder_outputs(self, features, feats_seq_len, num_unit_encoder, n_hidden_layer):
        self.keep_prob = tf.placeholder(tf.float32)
        
        cells_fw = [tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.GRUCell(num_unit_encoder), output_keep_prob=self.keep_prob) for _ in range(n_hidden_layer)]
        cells_bw = [tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.GRUCell(num_unit_encoder), output_keep_prob=self.keep_prob) for _ in range(n_hidden_layer)]
        
        # pooling
        prev_layer = features
        prev_seq_len = feats_seq_len
        for i, (cell_fw, cell_bw) in enumerate(zip(cells_fw, cells_bw)):
            with tf.variable_scope("cell_%d" % i):
                if i != 0:
                    #prev_layer = tf.expand_dims(outputs, axis=-1) # [batch, max_time, feats_dim, 1] = [batch, height, width, channels]
                    prev_layer = prev_layer[:,:,:,tf.newaxis]
                    prev_layer = tf.nn.avg_pool(prev_layer, ksize=(1,2,1,1), strides=(1,2,1,1), padding='VALID')
                    prev_layer = tf.squeeze(prev_layer, axis=[-1])
                    prev_seq_len = prev_seq_len // 2
                outputs, (state_fw, state_bw) = tf.nn.bidirectional_dynamic_rnn(cell_fw, cell_bw, prev_layer, sequence_length=prev_seq_len, dtype=tf.float32)
                prev_layer = tf.concat(outputs, 2)
                
        return prev_layer, prev_seq_len
        '''
        # concatenate
        prev_layer = features
        prev_seq_len = feats_seq_len
        for i, (cell_fw, cell_bw) in enumerate(zip(cells_fw, cells_bw)):
            with tf.variable_scope("cell_%d" % i):
                if i == n_hidden_layer - 1:
                    prev_even = prev_layer[:,0::2,:]
                    prev_odd = prev_layer[:,1::2,:]
                    prev_even = tf.cond(tf.less(tf.shape(prev_odd)[1], tf.shape(prev_even)[1]), lambda: prev_even[:,:-1,:], lambda: prev_even)
                    prev_layer = tf.concat([prev_even, prev_odd], 2)
                    prev_seq_len = prev_seq_len // 2
                
                outputs, (state_fw, state_bw) = tf.nn.bidirectional_dynamic_rnn(cell_fw, cell_bw, prev_layer, sequence_length=prev_seq_len, dtype=tf.float32)
                prev_layer = tf.concat(outputs, 2)
                           
        return prev_layer, prev_seq_len
        '''
    
    def _get_decoder_cell_and_init_state(self, mem_seq_len, num_unit_decoder, n_hidden_layer, memory, beam_width, model_type):
        
        batch_size = tf.shape(memory)[0]
        decoder_cell = tf.nn.rnn_cell.MultiRNNCell([tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.GRUCell(num_unit_decoder),
                                                                    output_keep_prob=self.keep_prob) for _ in range(n_hidden_layer-1)])
        #decoder_cell = tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.GRUCell(num_unit_decoder), output_keep_prob=self.keep_prob)

        if model_type == 'infer':
            memory = tf.contrib.seq2seq.tile_batch(memory, multiplier=beam_width)
            mem_seq_len = tf.contrib.seq2seq.tile_batch(mem_seq_len, multiplier=beam_width)
            batch_size = batch_size * beam_width

        attention_mechanism = tf.contrib.seq2seq.LuongAttention(num_unit_decoder, memory, mem_seq_len, scale=False)
        decoder_cell = tf.contrib.seq2seq.AttentionWrapper(decoder_cell, attention_mechanism, attention_layer_size=num_unit_decoder)
        decoder_initial_state = decoder_cell.zero_state(batch_size, tf.float32)
        
        return(decoder_cell, decoder_initial_state)
    
    def _compute_dynamic_decode(self, features, labels_in, feats_seq_len, labels_in_seq_len, num_unit_encoder, num_unit_decoder,
                        n_hidden_layer, num_classes, beam_width, model_type):
        embedding_decoder = tf.Variable(np.identity(num_classes), dtype=tf.float32, trainable=False)
        decoder_emb_inp = tf.nn.embedding_lookup(embedding_decoder, labels_in)
    
        memory, mem_seq_len = self._compute_encoder_outputs(features, feats_seq_len, num_unit_encoder, n_hidden_layer)
        with tf.variable_scope('decoder') as decoder_scope:
            decoder_cell, decoder_initial_state = self._get_decoder_cell_and_init_state(mem_seq_len, num_unit_decoder, n_hidden_layer,
                                                                                        memory, beam_width, model_type)
        
            output_layer = tf.contrib.keras.layers.Dense(num_classes, use_bias=False)

            if model_type in ['train', 'eval']:
                helper = tf.contrib.seq2seq.TrainingHelper(decoder_emb_inp, labels_in_seq_len, time_major=False)
                decoder = tf.contrib.seq2seq.BasicDecoder(decoder_cell, helper, initial_state=decoder_initial_state)
                outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(decoder, output_time_major=False, scope=decoder_scope)
                return output_layer(outputs.rnn_output)
            else: # model_type == 'infer'
                beam_decoder = tf.contrib.seq2seq.BeamSearchDecoder(decoder_cell, embedding_decoder, 
                                                    tf.fill([tf.shape(features)[0]], labels_sos_id), labels_eos_id,
                                                            decoder_initial_state, beam_width, output_layer=output_layer)
                decoded, _, final_seq_len = tf.contrib.seq2seq.dynamic_decode(beam_decoder, maximum_iterations=100,
                                                                              output_time_major=False, scope=decoder_scope)
                return decoded.predicted_ids # shape = [batch, max_time, beam_width]
    
    def _compute_loss(self, labels_out, labels_in_seq_len, logits):
        max_time = tf.shape(labels_out)[1]
        target_weights = tf.sequence_mask(labels_in_seq_len, max_time, dtype=logits.dtype)
        crossent = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels_out, logits=logits)
        return tf.reduce_sum(crossent * target_weights) / tf.to_float(tf.shape(logits)[0])
    
    def _get_update_step(self, loss, learning_rate):
        params = tf.trainable_variables()
        gradients = tf.gradients(loss, params)
        clipped_gradients, _ = tf.clip_by_global_norm(gradients, 5.0)
        optimizer = tf.train.AdamOptimizer(learning_rate)
        #optimizer = tf.train.GradientDescentOptimizer(learning_rate)
        return optimizer.apply_gradients(zip(clipped_gradients, params))
    
    def _compute_per(self, predicted_ids, labels_out, labels_eos_id, phn_61, phn_39, mapping):
        
        phn_61_tensor = tf.constant(phn_61, dtype=tf.string)
        phn_39_tensor = tf.constant(phn_39, dtype=tf.string)
        mapping_table = tf.contrib.lookup.HashTable(tf.contrib.lookup.KeyValueTensorInitializer(list(mapping.keys()), list(mapping.values())), default_value='')
        self.mapping_table_init = mapping_table.init
        
        def map_to_reduced_phn(p):
            val = mapping_table.lookup(phn_61_tensor[p])
            f1 = lambda: tf.to_int32(tf.reduce_min(tf.where(tf.equal(val, phn_39_tensor))))
            f2 = lambda: tf.to_int32(tf.reduce_min(tf.where(tf.equal(phn_61_tensor[p], phn_39_tensor))))
            return tf.cond(tf.not_equal(val, ''), f1, f2)
        
        indices = tf.to_int64(tf.where(tf.logical_and(tf.not_equal(predicted_ids[:,:,0], -1), tf.not_equal(predicted_ids[:,:,0], labels_eos_id))))
        vals = tf.to_int32(tf.gather_nd(predicted_ids[:,:,0], indices))
        shape = tf.to_int64(tf.shape(predicted_ids[:,:,0]))
        decoded_sparse = tf.SparseTensor(indices, vals, shape)
        labels_out_sparse = self._get_sparse_tensor(labels_out, labels_eos_id)
        
        decoded_reduced = tf.SparseTensor(decoded_sparse.indices, tf.map_fn(map_to_reduced_phn, decoded_sparse.values), decoded_sparse.dense_shape)
        labels_out_reduced = tf.SparseTensor(labels_out_sparse.indices, tf.map_fn(map_to_reduced_phn, labels_out_sparse.values), labels_out_sparse.dense_shape)
        
        return tf.reduce_sum(tf.edit_distance(decoded_reduced, labels_out_reduced, normalize=False)) , tf.to_float(tf.size(labels_out_reduced.values))

In [4]:
train_graph = tf.Graph()
dev_graph = tf.Graph()
test_graph = tf.Graph()

with train_graph.as_default():
    train_model = Model(batch_size, num_unit_encoder, num_unit_decoder, n_hidden_layer, feats_dim, num_classes, labels_sos_id, labels_eos_id,
                 learning_rate, file_type=TRAIN_FILE, model_type='train')
    initializer = tf.global_variables_initializer()

with dev_graph.as_default():
    dev_model = Model(batch_size, num_unit_encoder, num_unit_decoder, n_hidden_layer, feats_dim, num_classes, labels_sos_id, labels_eos_id,
                 file_type=DEV_FILE, model_type='eval')
with test_graph.as_default():
    test_model = Model(batch_size, num_unit_encoder, num_unit_decoder, n_hidden_layer, feats_dim, num_classes, labels_sos_id, labels_eos_id,
                 beam_width=10, phn_61=phn_61, phn_39=phn_39, mapping=mapping, file_type=TEST_FILE, model_type='infer')
    
train_sess = tf.Session(graph=train_graph)
train_sess.run(initializer)
#train_model.saver.restore(train_sess, './model/seq2seq_pyramidal/ckpt-26')

dev_sess = tf.Session(graph=dev_graph)

test_sess = tf.Session(graph=test_graph)
test_sess.run(test_model.mapping_table_init)

for epoch in range(epochs):
    train_sess.run(train_model.iterator_initializer)
    train_loss = []
    start = time.time()
    while True:
        try:
            _, cost = train_sess.run([train_model.update_step, train_model.loss], feed_dict={train_model.keep_prob: 0.6})
            #train_model.update_step.run(session=train_sess, feed_dict={train_model.keep_prob: 1.0})
            train_loss.append(cost)
        except tf.errors.OutOfRangeError:
            end = time.time()
            log = "Epoch {}/{}, \ntrain_loss={:.3f}, time={:.0f}s"
            print(log.format(epoch+1, epochs, np.mean(train_loss), end-start))
            checkpoint_path = train_model.saver.save(train_sess, checkpoints_path, global_step=epoch+1)
            
            dev_model.saver.restore(dev_sess, checkpoint_path)
            dev_sess.run(dev_model.iterator_initializer)
            start = time.time()
            dev_loss = []
            while True:
                try:
                    cost = dev_sess.run(dev_model.loss, feed_dict={dev_model.keep_prob: 1.0})
                    dev_loss.append(cost)
                except tf.errors.OutOfRangeError:
                    end = time.time()
                    log = "\tdev_loss={:.3f}, time={:.0f}s"
                    print(log.format(np.mean(dev_loss), end-start))
                    break
            
            test_model.saver.restore(test_sess, checkpoint_path)
            test_sess.run(test_model.iterator_initializer)
            test_per = []
            test_seq_len = []
            start = time.time()
            while True:
                try:
                    _per, seq_len = test_sess.run(test_model.per, feed_dict={test_model.keep_prob: 1.0})
                    test_per.append(_per)
                    test_seq_len.append(seq_len)
                except tf.errors.OutOfRangeError:
                    end = time.time()
                    log = '\t\ttest_per={:.3f}, time={:.0f}s'
                    print(log.format(sum(test_per)/sum(test_seq_len), end-start))
                    break
            
            break
            
train_sess.close()
dev_sess.close()
test_sess.close()

Epoch 1/100, 
train_loss=152.585, time=68s
INFO:tensorflow:Restoring parameters from ./model/seq2seq_pyramidal/ckpt-1
	dev_loss=147.106, time=4s
INFO:tensorflow:Restoring parameters from ./model/seq2seq_pyramidal/ckpt-1
		test_per=0.866, time=8s
Epoch 2/100, 
train_loss=147.624, time=68s
INFO:tensorflow:Restoring parameters from ./model/seq2seq_pyramidal/ckpt-2
	dev_loss=142.766, time=4s
INFO:tensorflow:Restoring parameters from ./model/seq2seq_pyramidal/ckpt-2
		test_per=1.721, time=14s
Epoch 3/100, 
train_loss=139.135, time=69s
INFO:tensorflow:Restoring parameters from ./model/seq2seq_pyramidal/ckpt-3
	dev_loss=119.970, time=4s
INFO:tensorflow:Restoring parameters from ./model/seq2seq_pyramidal/ckpt-3
		test_per=0.739, time=9s
Epoch 4/100, 
train_loss=116.477, time=69s
INFO:tensorflow:Restoring parameters from ./model/seq2seq_pyramidal/ckpt-4
	dev_loss=107.021, time=4s
INFO:tensorflow:Restoring parameters from ./model/seq2seq_pyramidal/ckpt-4
		test_per=0.784, time=10s
Epoch 5/100, 


INFO:tensorflow:Restoring parameters from ./model/seq2seq_pyramidal/ckpt-34
	dev_loss=30.613, time=4s
INFO:tensorflow:Restoring parameters from ./model/seq2seq_pyramidal/ckpt-34
		test_per=0.234, time=10s
Epoch 35/100, 
train_loss=28.338, time=69s
INFO:tensorflow:Restoring parameters from ./model/seq2seq_pyramidal/ckpt-35
	dev_loss=30.502, time=4s
INFO:tensorflow:Restoring parameters from ./model/seq2seq_pyramidal/ckpt-35
		test_per=0.235, time=10s
Epoch 36/100, 
train_loss=28.114, time=69s
INFO:tensorflow:Restoring parameters from ./model/seq2seq_pyramidal/ckpt-36
	dev_loss=30.547, time=4s
INFO:tensorflow:Restoring parameters from ./model/seq2seq_pyramidal/ckpt-36
		test_per=0.239, time=10s
Epoch 37/100, 
train_loss=26.942, time=69s
INFO:tensorflow:Restoring parameters from ./model/seq2seq_pyramidal/ckpt-37
	dev_loss=30.568, time=4s
INFO:tensorflow:Restoring parameters from ./model/seq2seq_pyramidal/ckpt-37
		test_per=0.238, time=10s
Epoch 38/100, 
train_loss=26.475, time=69s
INFO:ten

KeyboardInterrupt: 

In [ ]:
infer_train_graph = tf.Graph()
infer_test_graph = tf.Graph()

with infer_train_graph.as_default():
    infer_train_model = Model(batch_size*2, num_unit_encoder, num_unit_decoder, n_hidden_layer, feats_dim, num_classes, labels_sos_id, labels_eos_id,
                 beam_width=10, phn_61=phn_61, phn_39=phn_39, mapping=mapping, file_type=TRAIN_FILE, model_type='infer')
    #infer_train_initializer = tf.global_variables_initializer()
with infer_test_graph.as_default():
    infer_test_model = Model(batch_size*2, num_unit_encoder, num_unit_decoder, n_hidden_layer, feats_dim, num_classes, labels_sos_id, labels_eos_id,
                 beam_width=10, phn_61=phn_61, phn_39=phn_39, mapping=mapping, file_type=TEST_FILE, model_type='infer')
    #infer_test_initializer = tf.global_variables_initializer()

infer_train_sess = tf.Session(graph=infer_train_graph)
infer_train_sess.run(infer_train_model.mapping_table_init)
#infer_train_sess.run(infer_train_initializer)
infer_test_sess = tf.Session(graph=infer_test_graph)
infer_test_sess.run(infer_test_model.mapping_table_init)
#infer_test_sess.run(infer_test_initializer)

#last_checkpoints = train_model.saver.last_checkpoints[1]

for ckpt_path in train_model.saver.last_checkpoints:
    infer_train_model.saver.restore(infer_train_sess, ckpt_path)
    infer_train_sess.run(infer_train_model.iterator_initializer)
    infer_train_per = []
    start = time.time()
    while True:
        try:
            _per = infer_train_sess.run(infer_train_model.per, feed_dict={infer_train_model.keep_prob: 1.0})
            infer_train_per.append(_per)
        except tf.errors.OutOfRangeError:
            end = time.time()
            log = "{} train_per={:.3f}, time={:.0f}s"
            print(log.format(ckpt_path, np.mean(infer_train_per), end-start))
            break

    infer_test_model.saver.restore(infer_test_sess, ckpt_path)
    infer_test_sess.run(infer_test_model.iterator_initializer)
    infer_test_per = []
    start = time.time()
    while True:
        try:
            _per = infer_test_sess.run(infer_test_model.per, feed_dict={infer_test_model.keep_prob: 1.0})
            infer_test_per.append(_per)
        except tf.errors.OutOfRangeError:
            end = time.time()
            log = "{} test_per={:.3f}, time={:.0f}s"
            print(log.format(ckpt_path, np.mean(infer_test_per), end-start))
            break
        
infer_train_sess.close()
infer_test_sess.close()